# Acquisition Functions
> Wrapping wrappers wraps wrappers.

In [ ]:
# default_exp acquisition_functions

Import modules and functions were are going to use.

In [ ]:
# exports

from dataclasses import dataclass

import numpy as np
import torch
import torch.utils.data

from batchbald_redux.batchbald import (
    CandidateBatch,
    get_bald_scores,
    get_batch_bald_batch,
    get_batch_coreset_bald_batch,
    get_batch_eig_batch,
    get_batch_eval_bald_batch,
    get_coreset_bald_scores,
    get_coreset_bald_scores_from_predictions,
    get_coreset_eig_bald_scores,
    get_coreset_eig_scores,
    get_eig_scores,
    get_eval_bald_scores,
    get_real_naive_epig_scores,
    get_sampled_tempered_scorers,
    get_sieve_bald_batch,
    get_thompson_bald_batch,
    get_top_k_scorers,
    get_top_random_scorers,
)
from batchbald_redux.trained_model import TrainedModel

In [ ]:
# exports


@dataclass
class CandidateBatchComputer:
    acquisition_size: int

    def compute_candidate_batch(
        self, model: TrainedModel, pool_loader: torch.utils.data.DataLoader, device
    ) -> CandidateBatch:
        pass

In [ ]:
# exports


@dataclass
class Random(CandidateBatchComputer):
    def compute_candidate_batch(
        self, model: TrainedModel, pool_loader: torch.utils.data.DataLoader, device
    ) -> CandidateBatch:
        num_pool_samples = len(pool_loader.dataset)
        indices = np.random.choice(num_pool_samples, size=self.acquisition_size, replace=False)
        candidate_batch = CandidateBatch([0.0] * self.acquisition_size, indices)
        return candidate_batch

In [ ]:
# exports


@dataclass
class PoolScorerCandidateBatchComputer(CandidateBatchComputer):
    num_pool_samples: int

    def compute_candidate_batch(
        self, model: TrainedModel, pool_loader: torch.utils.data.DataLoader, device
    ) -> CandidateBatch:
        log_probs_N_K_C = model.get_log_probs_N_K_C(pool_loader, self.num_pool_samples, device, "cpu")

        return self.get_candidate_batch(log_probs_N_K_C, device)

    def get_candidate_batch(self, log_probs_N_K_C, device) -> CandidateBatch:
        raise NotImplementedError()

In [ ]:
# exports


@dataclass
class _BALD(PoolScorerCandidateBatchComputer):
    def get_candidate_batch(self, log_probs_N_K_C, device) -> CandidateBatch:
        # Evaluate BALD scores
        scores_N = get_bald_scores(log_probs_N_K_C, dtype=torch.double, device=device)

        candidate_batch = self.extract_candidates(scores_N)

        return candidate_batch

    def extract_candidates(self, scores_N) -> CandidateBatch:
        raise NotImplementedError()


@dataclass
class BALD(_BALD):
    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_top_k_scorers(scores_N, batch_size=self.acquisition_size)


@dataclass
class TemperedBALD(_BALD):
    temperature: float

    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_sampled_tempered_scorers(scores_N, batch_size=self.acquisition_size, temperature=self.temperature)


@dataclass
class SoftmaxBALD(_BALD):
    temperature: float

    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_sampled_tempered_scorers(
            scores_N.exp(), batch_size=self.acquisition_size, temperature=self.temperature
        )


@dataclass
class RandomBALD(_BALD):
    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_top_random_scorers(scores_N, batch_size=self.acquisition_size, num_classes=10)


@dataclass
class ThompsonBALD(PoolScorerCandidateBatchComputer):
    def get_candidate_batch(self, log_probs_N_K_C, device) -> CandidateBatch:
        candidate_batch = get_thompson_bald_batch(
            log_probs_N_K_C,
            batch_size=self.acquisition_size,
            dtype=torch.double,
            device=device,
        )
        return candidate_batch

In [ ]:
# exports


@dataclass
class SieveBALD(PoolScorerCandidateBatchComputer):
    def get_candidate_batch(self, log_probs_N_K_C, device) -> CandidateBatch:
        # Evaluate BALD scores
        candidate_batch = get_sieve_bald_batch(
            log_probs_N_K_C,
            batch_size=self.acquisition_size,
            dtype=torch.double,
            device=device,
        )
        return candidate_batch

In [ ]:
# exports


@dataclass
class BatchBALD(PoolScorerCandidateBatchComputer):
    num_samples: int = 1000000

    def get_candidate_batch(self, log_probs_N_K_C, device) -> CandidateBatch:
        # Evaluate BALD scores
        candidate_batch = get_batch_bald_batch(
            log_probs_N_K_C,
            batch_size=self.acquisition_size,
            num_samples=self.num_samples,
            dtype=torch.double,
            device=device,
        )
        return candidate_batch

In [ ]:
# exports


@dataclass
class CoreSetPoolScorerCandidateBatchComputer(CandidateBatchComputer):
    num_pool_samples: int

    def compute_candidate_batch(
        self, model: TrainedModel, pool_loader: torch.utils.data.DataLoader, device
    ) -> CandidateBatch:
        log_probs_N_K_C, labels_N = model.get_log_probs_N_K_C_labels_N(
            pool_loader, self.num_pool_samples, device, "cpu"
        )

        return self.get_candidate_batch(log_probs_N_K_C, labels_N, device)

    def get_candidate_batch(self, log_probs_N_K_C, labels_N, device) -> CandidateBatch:
        raise NotImplementedError()

In [ ]:
# exports


@dataclass
class _CoreSetBALD(CoreSetPoolScorerCandidateBatchComputer):
    def get_candidate_batch(self, log_probs_N_K_C, labels_N, device) -> CandidateBatch:
        if len(labels_N.shape) == 1:
            scores_N = get_coreset_bald_scores(log_probs_N_K_C, labels_N, dtype=torch.double, device=device)
        else:
            scores_N = get_coreset_bald_scores_from_predictions(
                log_probs_N_K_C, labels_N, dtype=torch.double, device=device
            )

        candidate_batch = self.extract_candidates(scores_N)

        return candidate_batch

    def extract_candidates(self, scores_N) -> CandidateBatch:
        raise NotImplementedError()


@dataclass
class CoreSetBALD(_CoreSetBALD):
    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_top_k_scorers(scores_N, batch_size=self.acquisition_size)


@dataclass
class TemperedCoreSetBALD(_CoreSetBALD):
    temperature: float

    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_sampled_tempered_scorers(scores_N, batch_size=self.acquisition_size, temperature=self.temperature)

In [ ]:
# exports


@dataclass
class BatchCoreSetBALD(CoreSetPoolScorerCandidateBatchComputer):
    def get_candidate_batch(self, log_probs_N_K_C, labels_N, device) -> CandidateBatch:
        candidate_batch = get_batch_coreset_bald_batch(
            log_probs_N_K_C,
            labels_N,
            batch_size=self.acquisition_size,
            dtype=torch.double,
            device=device,
        )
        return candidate_batch

In [ ]:
# exports


@dataclass
class EvalDatasetBatchComputer:
    acquisition_size: int

    def compute_candidate_batch(
        self,
        model: TrainedModel,
        pool_loader: torch.utils.data.DataLoader,
        eval_loader: torch.utils.data.DataLoader,
        device,
    ) -> CandidateBatch:
        pass

In [ ]:
# exports


@dataclass
class EvalModelBatchComputer:
    acquisition_size: int

    def compute_candidate_batch(
        self, model: TrainedModel, eval_model: TrainedModel, pool_loader: torch.utils.data.DataLoader, device
    ) -> CandidateBatch:
        pass

NameError: name 'dataclass' is not defined

In [ ]:
# exports


@dataclass
class EvaluationPoolScorerCandidateBatchComputer(EvalModelBatchComputer):
    num_pool_samples: int

    def compute_candidate_batch(
        self, model: TrainedModel, eval_model: TrainedModel, pool_loader: torch.utils.data.DataLoader, device
    ) -> CandidateBatch:
        log_probs_N_K_C = model.get_log_probs_N_K_C(pool_loader, self.num_pool_samples, device, "cpu")
        log_eval_probs_N_K_C = eval_model.get_log_probs_N_K_C(pool_loader, self.num_pool_samples, device, "cpu")

        return self.get_candidate_batch(log_probs_N_K_C, log_eval_probs_N_K_C, device)

    def get_candidate_batch(self, log_probs_N_K_C, log_eval_probs_N_K_C, device) -> CandidateBatch:
        raise NotImplementedError()

In [ ]:
# exports


@dataclass
class _EvalBALD(EvaluationPoolScorerCandidateBatchComputer):
    def get_candidate_batch(self, log_probs_N_K_C, log_eval_probs_N_K_C, device) -> CandidateBatch:
        scores_N = get_eval_bald_scores(log_probs_N_K_C, log_eval_probs_N_K_C, dtype=torch.double, device=device)

        candidate_batch = self.extract_candidates(scores_N)

        return candidate_batch

    def extract_candidates(self, scores_N) -> CandidateBatch:
        raise NotImplementedError()


@dataclass
class EvalBALD(_EvalBALD):
    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_top_k_scorers(scores_N, batch_size=self.acquisition_size)


@dataclass
class TemperedEvalBALD(_EvalBALD):
    temperature: float

    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_sampled_tempered_scorers(scores_N, batch_size=self.acquisition_size, temperature=self.temperature)

In [ ]:
# exports


@dataclass
class _EPIG(EvalDatasetBatchComputer):
    num_pool_samples: int

    def compute_candidate_batch(
        self,
        model: TrainedModel,
        eval_loader: torch.utils.data.DataLoader,
        pool_loader: torch.utils.data.DataLoader,
        device,
    ) -> CandidateBatch:
        log_probs_N_K_C = model.get_log_probs_N_K_C(pool_loader, self.num_pool_samples, device, "cpu")
        log_eval_probs_N_K_C = model.get_log_probs_N_K_C(eval_loader, self.num_pool_samples, device, "cpu")

        scores_N = get_real_naive_epig_scores(
            pool_log_probs_N_K_C=log_probs_N_K_C,
            eval_log_probs_E_K_C=log_eval_probs_N_K_C,
            dtype=torch.double,
            device=device,
        )

        candidate_batch = self.extract_candidates(scores_N)

        return candidate_batch

    def extract_candidates(self, scores_N) -> CandidateBatch:
        raise NotImplementedError()


@dataclass
class EPIG(_EPIG):
    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_top_k_scorers(scores_N, batch_size=self.acquisition_size)


@dataclass
class SoftmaxEPIG(_EPIG):
    temperature: float

    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_sampled_tempered_scorers(scores_N, batch_size=self.acquisition_size, temperature=self.temperature)

In [ ]:
# exports


@dataclass
class BatchEvalBALD(EvaluationPoolScorerCandidateBatchComputer):
    num_samples: int = 1000000

    def get_candidate_batch(self, log_probs_N_K_C, log_eval_probs_N_K_C, device) -> CandidateBatch:
        candidate_batch = get_batch_eval_bald_batch(
            log_probs_N_K_C,
            log_eval_probs_N_K_C,
            batch_size=self.acquisition_size,
            num_samples=self.num_samples,
            dtype=torch.double,
            device=device,
        )
        return candidate_batch

In [ ]:
# exports


@dataclass
class _EIG(EvaluationPoolScorerCandidateBatchComputer):
    def get_candidate_batch(self, log_probs_N_K_C, log_eval_probs_N_K_C, device) -> CandidateBatch:
        scores_N = get_eig_scores(log_probs_N_K_C, log_eval_probs_N_K_C, dtype=torch.double, device=device)

        candidate_batch = self.extract_candidates(scores_N)

        return candidate_batch

    def extract_candidates(self, scores_N) -> CandidateBatch:
        raise NotImplementedError()


@dataclass
class EIG(_EIG):
    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_top_k_scorers(scores_N, batch_size=self.acquisition_size)


@dataclass
class BatchEIG(EvaluationPoolScorerCandidateBatchComputer):
    num_samples: int = 1000000

    def get_candidate_batch(self, log_probs_N_K_C, log_eval_probs_N_K_C, device) -> CandidateBatch:
        candidate_batch = get_batch_eig_batch(
            log_probs_N_K_C,
            log_eval_probs_N_K_C,
            batch_size=self.acquisition_size,
            num_samples=self.num_samples,
            dtype=torch.double,
            device=device,
        )
        return candidate_batch


@dataclass
class TemperedEIG(_EIG):
    temperature: float

    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_sampled_tempered_scorers(scores_N, batch_size=self.acquisition_size, temperature=self.temperature)

In [ ]:
# exports


@dataclass
class CoreSetEvaluationPoolScorerCandidateBatchComputer(EvalModelBatchComputer):
    num_pool_samples: int

    def compute_candidate_batch(
        self, model: TrainedModel, eval_model: TrainedModel, pool_loader: torch.utils.data.DataLoader, device
    ) -> CandidateBatch:
        training_log_probs_N_K_C, training_labels_N = model.get_log_probs_N_K_C_labels_N(
            pool_loader, self.num_pool_samples, device, "cpu"
        )
        eval_log_probs_N_K_C, eval_labels_N = eval_model.get_log_probs_N_K_C_labels_N(
            pool_loader, self.num_pool_samples, device, "cpu"
        )

        # With high probability, this ensures that we are not shuffling the batches.
        assert torch.equal(training_labels_N, eval_labels_N)

        return self.get_candidate_batch(training_log_probs_N_K_C, eval_log_probs_N_K_C, training_labels_N, device)

    def get_candidate_batch(self, training_log_probs_N_K_C, eval_log_probs_N_K_C, labels_N, device) -> CandidateBatch:
        raise NotImplementedError()

In [ ]:
# exports


@dataclass
class _CoreSetPIGBALD(CoreSetEvaluationPoolScorerCandidateBatchComputer):
    def get_candidate_batch(self, training_log_probs_N_K_C, eval_log_probs_N_K_C, labels_N, device) -> CandidateBatch:
        scores_N = get_coreset_eig_bald_scores(
            training_log_probs_N_K_C=training_log_probs_N_K_C,
            eval_log_probs_N_K_C=eval_log_probs_N_K_C,
            labels_N=labels_N,
            dtype=torch.double,
            device=device,
        )
        candidate_batch = self.extract_candidates(scores_N)

        return candidate_batch

    def extract_candidates(self, scores_N) -> CandidateBatch:
        raise NotImplementedError()


@dataclass
class CoreSetPIGBALD(_CoreSetPIGBALD):
    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_top_k_scorers(scores_N, batch_size=self.acquisition_size)


@dataclass
class TemperedCoreSetPIGBALD(_CoreSetPIGBALD):
    temperature: float

    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_sampled_tempered_scorers(scores_N, batch_size=self.acquisition_size, temperature=self.temperature)

In [ ]:
# exports


@dataclass
class _CoreSetPIG(CoreSetEvaluationPoolScorerCandidateBatchComputer):
    def get_candidate_batch(self, training_log_probs_N_K_C, eval_log_probs_N_K_C, labels_N, device) -> CandidateBatch:
        scores_N = get_coreset_eig_scores(
            training_log_probs_N_K_C=training_log_probs_N_K_C,
            eval_log_probs_N_K_C=eval_log_probs_N_K_C,
            labels_N=labels_N,
            dtype=torch.double,
            device=device,
        )
        candidate_batch = self.extract_candidates(scores_N)

        return candidate_batch

    def extract_candidates(self, scores_N) -> CandidateBatch:
        raise NotImplementedError()


@dataclass
class CoreSetPIG(_CoreSetPIG):
    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_top_k_scorers(scores_N, batch_size=self.acquisition_size)


@dataclass
class TemperedCoreSetPIG(_CoreSetPIG):
    temperature: float

    def extract_candidates(self, scores_N) -> CandidateBatch:
        return get_sampled_tempered_scorers(scores_N, batch_size=self.acquisition_size, temperature=self.temperature)